#### Servomotor controlado por pagina WEB usando ESP8266 

Se utiliza el circuito de la figura
<img src="img/nodemcu_servo.png" alt="Imagen"></img>
El programa tiene los siguientes archivos:

- config.h
- actuador.hpp
- WIFI.hpp
- server.hpp
- archivo principal


##### config.h

```
const char Mi_pagina[] PROGMEM = R"=====(
<!DOCTYPE html>  <html> 
<head><meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0, user-scalable=no\">
<title>CONTROL DE servomotor</title>
</head>

<body>  
<h1>Servomotor </h1><br>

<form method="post" action="/angulo">
Angulo:
<input type="text" name="angul" size="20">
<br>
<input type="submit" value="enviar">
</form>

</body>
</html>
)=====";

const char* ssid="########";
const char* password="********";
```

##### actuador.hpp

Crea el objeto servo y la funcion iniciarServo que configura el servomotor

```
Servo uno;

void iniciarServo()
{
  pinMode(D1, OUTPUT);
  uno.attach(D1);
}
```

In [ ]:
#### wifi.hpp

Configura y conecta la red wifi


```
void configuracion_wifi_STA()
{
  WiFi.begin(ssid, password);
  Serial.println(" ");
  Serial.println("Conectando");
  while (WiFi.status() != WL_CONNECTED)
  {
    delay(500);
    Serial.print(".");
  }
  Serial.println();
  Serial.print("Conectado, dirección IP: ");
  Serial.println(WiFi.localIP());
}

void configuracion_wifi_AP()
{
  WiFi.mode(WIFI_AP);  
  while(!WiFi.softAP(ssid, password))  
  {  
   Serial.println(".");  
    delay(100);  
  }  
  Serial.print("Iniciado AP ");  
  Serial.println(ssid);  
  Serial.print("IP address:\t");  
  Serial.println(WiFi.softAPIP());  
}
```


In [ ]:
#### server.hpp

Configura, inicia y controla el servidor Web, las funciones son:
- controlRaiz: Carga la pagina principal
- controlAngulo Controla el servomotor desde la pagina
- iniciarServidor:  Contorla la pagina con sus distintas funciones


```
ESP8266WebServer server(80); //Server on port 80

void controlRaiz() {
  int i;
  Serial.println("Pagina original ");
  String s = Mi_pagina; //Read HTML contents
  server.send(200, "text/html", s); //Send web page
}

void controlAngulo() {
  String a;
  int valor;
  a=server.arg("angul");
  valor=a.toInt();
  uno.write(valor);
  Serial.println(server.arg("angul"));
  Serial.println(valor);
  String s = Mi_pagina; //Read HTML contents
  delay(200);
  server.send(200, "text/html", s); //Send web page
}
void iniciarServidor()
{
  server.on("/", controlRaiz);
  server.on("/angulo", controlAngulo);
  server.begin();
}
```

#### Archivo principal

Carga todas las librerias e inicializa el dispositivo, inicializa wifi y el servidorweb

```
void setup()  
{   
Serial.begin(115200);  
pinMode(LED_BUILTIN,OUTPUT);
Serial.println("Hola");
iniciarServo();  
configuracion_wifi_STA();  
iniciar_servidor();
}  

void loop() {
  server.handleClient();
  }
```

### Servidor NPT

El Protocolo de Tiempo por Red o Network Time Protocol (NTP) es un protocolo de Internet utilizado para sincronizar los relojes internos de sistemas informáticos (tales como computadoras personales o servidores corporativos). La sincronización esporádica del reloj interno de las computadoras (por ejemplo, dos veces por semana) evita desfasajes en el tiempo local de las mismas. Dichos desfasajes de tiempo pueden ocasionar serios inconvenientes durante el intercambio de datos con otros sistemas.
#### Libreria
Se puede incluir este servicio utilizando la libreria NTPClient.h

Para incluir la librería se precede de la siguiente manera:

En el IDE Arduino, ir a > Libreria > Administrar bibliotecas. Se abre el Gestor de Bibliotecas. Se Busca NTPClient y aparece una de Fabrice Weinber. Instalar
#### Uso

Se crea un objeto ntpUDP de la clase WiFiUDP
``` WiFiUDP ntpUDP; ```

Se crea un objeto tiempo de la clase NTPClient: el cual es el encargado de suministrar la hora.

```NTPClient tiempo(ntpUDP, "pool.ntp.org", utcOffsetInSeconds); ```

Se inicializa el objeto tiempo en la funcion *setup()*

``` tiempo.begin() ``` 

#### Funciones

|Nombre |	sintaxis |	Funcion|
|---    |---         |---      |
|NTPClient |	NTPClient tiempo(ntpUDP, "pool.ntp.org", utcOffsetInSeconds); |	crea un objeto tiempo de la clase NTPClient|
| begin |	tiempo.begin() |	Inicializa el objeto tiempo|
|update |	tiempo.update()| 	mira la hora en Internet|
|getDay |	dia_sem[tiempo.getDay()] |	Muestra el dia de la semana|
|getHour| 	tiempo.getHour() |	Presenta la hora|
|getMinutes| 	tiempo.getMinutes() |	Presenta los minutos|
|getSecond| 	tiempo.getSecond ()| 	Presenta los segundos|
|getFormattedTime| 	tiempo.getFormattedTime() |	Presenta todos los datos|


#### Ejemplo

```
#include <NTPClient.h>  
#include <WiFiUdp.h>  

const char *ssid     = "red";  
const char *password = "password";  

const long utcOffsetInSeconds = 10800;  

char daysOfTheWeek[7][12] = {"Domingo", "Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado"};  

// Define NTP Client to get time  
WiFiUDP ntpUDP;  
NTPClient timeClient(ntpUDP, "pool.ntp.org", utcOffsetInSeconds);  

void setup(){  
  Serial.begin(115200);  
  timeClient.begin();  
}

void loop() {  
  timeClient.update();  

  Serial.print(daysOfTheWeek[timeClient.getDay()]);  
  Serial.print(", ");  
  Serial.print(timeClient.getHours());  
  Serial.print(":");  
  Serial.print(timeClient.getMinutes());  
  Serial.print(":");  
  Serial.println(timeClient.getSeconds());  
  //Serial.println(timeClient.getFormattedTime());  

  delay(1000);  
}
```

In [ ]:
SPIFF

